In [3]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

from src.folder_handler import *
from src.cort_processor import *
from src.tdt_support import *
from src.plotter import *
from src.decoders import *
import pickle

# load data
data should be a single folder. Inside there should be a 'angles' (with angles), 'pose-3d', and 'tdt' folders named exactly that

In [ ]:
session519 = CortProcessor('/home/diya/Documents/rat-fes/data/rat-annie/5-19') #insert name of your folder here

In [ ]:
session519.tdt_data[0]

# process data
here, we crop our data, turn our raw data into rates, and we resample our angles to be same times as binned spikes

In [6]:
session512.process?


Signature:
session512.process(
    manual_crop_list=None,
    threshold_multiplier=3.0,
    binsize=0.05,
)
Docstring:
1) takes raw neural data, then bandpass+notch filters, then extracts
spikes using threshold_multiplier, then bins into firing rates with bin
size
2) takes coordinates/angles, then resamples at 20 HZ
3) saves all under CortHandler.data

crop is automatically read from config.yaml file, unless manual crop
list is set.
File:      ~/Documents/rat-fes/src/cort_processor.py
Type:      method


In [ ]:
rates519, angles519 = session519.process()

In [ ]:
session519.data.keys()

# loading data using pickles (and process_cort.py in scripts)

in order to process data using command line, navigate to /scripts, and run 'python3 process_cort.py *path_to_folder'

In [4]:
with open('/home/diya/Documents/rat-fes/data/pickles/rat-annie_5-12_session.pkl', 'rb') as inp:
    session512 = pickle.load(inp)

# other pipeline processing, like getting toe height

In [ ]:
toe_height = session519.process_toe_height()

In [ ]:
session519.data['toe_height']

# and now some decoding

## angles

In [ ]:
h_angles, vaf_angles, best_x_angles, best_y_angles = session519.decode_angles()

In [ ]:
np.average(vaf_angles, 1)

In [ ]:
plot_wiener_filter_predic(best_x_angles, best_y_angles[:,3], h_angles[:,3])

## toe_height

In [ ]:
h_toe, vaf_toe, best_x_toe, best_y_toe = session519.decode_toe_height()

In [ ]:
plot_wiener_filter_predic(best_x_toe, best_y_toe, h_toe)

# or dividing into gaits, then plotting sorted raster.

In [ ]:
gait_indices, avg_gait_samples = session519.get_gait_indices()

In [ ]:
X_gait, Y_gait = session519.divide_into_gaits()

In [ ]:
#proc_rates, proc_angles = session512.remove_bad_gaits()

In [ ]:
df = session519.neuron_tuning()

In [ ]:
plot_raster(df)